# hw 2

Скачиваем данные, убираем отзывы без текста, смотрим на них

In [3]:
import pandas as pd
import gzip
import json
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Grocery_and_Gourmet_Food_5.json.gz')

In [10]:
df = df.dropna(subset=['reviewText'])

In [88]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,None,None,None
1,5.0,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,None,None,None
2,5.0,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,None,None,None
3,5.0,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,None,None,None
4,5.0,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,None,None,None


## 1. 

1 Получить название товаров из метадаты: "title" для точного названия,что-то из "categories" для его замены(например iphone будет в категории phones и его могут так называть в отзывах). После этого можно попрбовать расширить через близость с словами в самом отзыве.

\+ сразу получаем большой словарь товаров

\- нужны метаданные

\- название товара не всегда отражает его сущность (Пример: из названия Hungry Hungry Hippos непонятно, что это игра)

2 Написать словарь самому и попробовать расширить его поиском близких слова в тексте обзора.

\+ получаем точный словарь

\- нужно собирать его вручную, что в случае с категорией с множеством подкатегорий может быть затруднительно

3 Собирать упоминания через правила для контекстов. 

\+ собираем упоминания непосредственно из отзывов

\+ сразу собираем контекст

\- сложности с составлением правил

\- невозможно учесть в правилах абсолютно все случаи

Я буду реализовывать третий способ

In [50]:
# убираем все короткие предложения
review = df.reviewText
print(len(review))
review = [i for i in review if len(i.split()) > 3]
print(len(review))

1143470
982713


Я придумал два правила:
+ глаголы, связанные с едой + определенный артикль + возможные прилагательные + х
+ слова типа вкус, цена, упаковка + "of" + -||-

В первое правило иногда попадало существительное из начала второго, поэтому я передаю список исключений для х

In [182]:
import spacy
import re
import random as rnd
import nltk
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
of_as_ent = ["pack", "can", "bottle", "jar","bag"]
x_of_list = ["taste","flavor","price","freshness","packaging","texture","type","kind","squeeze","sweetness","aroma","feeling"] + of_as_ent
exception_list = ["convenience","fact","idea","brand"] + x_of_list + of_as_ent
verb_pattern = [{"LEMMA": {"IN": ["like","eat","drink","cook","buy"]},"POS": "VERB"},{"POS": "DET","LEMMA":{"IN": ["this", "the","these"]}},{"POS": "ADJ","OP":"*"},{"POS": "NOUN","LEMMA": {"NOT_IN": exception_list}}]
of_pattern = [{"LEMMA": {"IN": x_of_list},"POS": "NOUN"},{"POS":"ADP", "LEMMA":"of"},{"POS": "DET","LEMMA":{"IN": ["this", "the","these"]}},{"POS": "ADJ","OP":"*"},{"POS": "NOUN", "LEMMA":{"NOT_IN": exception_list}}]

In [183]:
entities = []
matcher = Matcher(nlp.vocab)
matcher.add("ment", [verb_pattern, of_pattern])
count = 0
bigrams_left = []
bigrams_right = []
for i in review[:100000]:
    doc = nlp(i)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span   
        entity = span.text.lower().split()[-1]
        entities.append(entity)
        finder = nltk.BigramCollocationFinder.from_words(nltk.word_tokenize(i))
        word_filter = lambda w1, w2: entity not in (w1, w2)
        finder.apply_ngram_filter(word_filter)
        bigrams_list = finder.ngram_fd.items()
        bigrams_left += [i[0] for i in bigrams_list if i[0][1]==entity and i[0][0].isalnum() and not i[0][0].lower() in ["the","this","these","a","an"]]
        bigrams_right += [i[0] for i in bigrams_list if i[0][0]==entity and i[0][1].isalnum() and not i[0][1].lower() in ["the","this","these","a","an"]]
    count+=1
    if count%10000 == 0:
        print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [187]:
ent_set = set(entities)
list(ent_set)[:100]

['waxy',
 'past',
 'ingredient',
 'falafel',
 'bottom',
 'selections',
 'products',
 'fig',
 'air',
 'candies',
 'gumbo',
 'loss',
 'dish',
 'batch',
 'crme',
 'subscription',
 'brew',
 'agave',
 'bit',
 'design',
 'tea',
 'container',
 'topping',
 'lollipops',
 'skin',
 'caffeine',
 'deal',
 'dark',
 'stores',
 'bowl',
 'roast',
 'lots',
 'poisons',
 'bacon',
 'layer',
 'anytime',
 'version',
 'name',
 'proportion',
 'convinience',
 'peppers',
 'fennel',
 'shoulder',
 'crumbs',
 'lentils',
 'bonus',
 'salino',
 'tapioca',
 'loaf',
 'basket',
 'availability',
 'mushrooms',
 'cammommile',
 'servings',
 'piece',
 'hondashi',
 'equivalent',
 'miso',
 'oil',
 'lifesavers',
 'orient',
 'breads',
 'frustration',
 'licorice',
 'beauties',
 'canister',
 'smoke',
 'lemon',
 'splurge',
 'set',
 'spice',
 'eco',
 'salt',
 'puffs',
 'store',
 'cheesy',
 'drops',
 'banana',
 'world',
 'breakfast',
 'loaves',
 'use',
 'tasting',
 'butterscotch',
 'ways',
 'quantities',
 'olives',
 'run',
 'formula',

In [185]:
bigrams_left[:100]

[('loose', 'tea'),
 ('black', 'tea'),
 ('final', 'product'),
 ('black', 'tea'),
 ('loose', 'tea'),
 ('or', 'loose'),
 ('curry', 'spice'),
 ('rock', 'salt'),
 ('particular', 'power'),
 ('of', 'power'),
 ('whole', 'bar'),
 ('single', 'bar'),
 ('enjoys', 'sprinkles'),
 ('same', 'container'),
 ('giant', 'container'),
 ('first', 'blend'),
 ('large', 'package'),
 ('stronger', 'peppermint'),
 ('better', 'gum'),
 ('occasion', 'gum'),
 ('better', 'gum'),
 ('occasion', 'gum'),
 ('much', 'gum'),
 ('bubble', 'gum'),
 ('espresso', 'coffee'),
 ('having', 'coffee'),
 ('whole', 'bean'),
 ('my', 'coffee'),
 ('excellent', 'coffee'),
 ('BLAH', 'coffee'),
 ('chocolate', 'peppermint'),
 ('love', 'peppermint'),
 ('white', 'chocolate'),
 ('even', 'spray'),
 ('gold', 'spray'),
 ('to', 'spray'),
 ('decorate', 'cookies'),
 ('jerky', 'gun'),
 ('various', 'jerky'),
 ('make', 'jerky'),
 ('of', 'spice'),
 ('coconut', 'one'),
 ('rice', 'vinegar'),
 ('hot', 'classic'),
 ('wasabi', 'product'),
 ('robust', 'oil'),
 ('o

In [186]:
bigrams_right[:100]

[('tea', 'after'),
 ('tea', 'as'),
 ('tea', 'rather'),
 ('tea', 'bags'),
 ('loose', 'from'),
 ('kaffir', 'lime'),
 ('spice', 'seems'),
 ('tea', 'at'),
 ('tea', 'for'),
 ('tea', 'for'),
 ('tea', 'when'),
 ('tea', 'has'),
 ('breathe', 'easy'),
 ('tea', 'bag'),
 ('tea', 'will'),
 ('tea', 'was'),
 ('product', 'in'),
 ('product', 'in'),
 ('salt', 'except'),
 ('salt', 'along'),
 ('sounds', 'of'),
 ('power', 'bars'),
 ('bar', 'so'),
 ('bar', 'was'),
 ('bar', 'in'),
 ('candy', 'taste'),
 ('shapes', 'that'),
 ('sprinkles', 'for'),
 ('sprinkles', 'on'),
 ('sprinkles', 'were'),
 ('sprinkles', 'nut'),
 ('variety', 'rather'),
 ('container', 'at'),
 ('container', 'which'),
 ('blend', 'I'),
 ('package', 'on'),
 ('gum', 'is'),
 ('peppermint', 'or'),
 ('bubblegum', 'flavor'),
 ('gum', 'at'),
 ('gum', 'was'),
 ('cinnamint', 'is'),
 ('cinnamint', 'lasts'),
 ('gum', 'at'),
 ('coffee', 'when'),
 ('coffee', 'at'),
 ('bean', 'and'),
 ('coffee', 'for'),
 ('coffee', 'is'),
 ('cereal', 'is'),
 ('peppermint', 'b